In [3]:
import tweepy
import json
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim.downloader as gensimapi
from nltk.tokenize import TreebankWordTokenizer
from emoji import UNICODE_EMOJI
import unicodedata as ud
import functools
import operator
import re
import numpy as np
!pip install wordfreq
from wordfreq import word_frequency
import emoji

consumer_key = "idjkP1aobw1UQd8xZ9RYiY5CZ"
consumer_secret = "jZFXsLJRtvR4pQvmuTJ94mnr1TJ0tYz1w4s0XI5TpR4U5tEnXe"
access_token = "1001251273981677568-5SxiGu3SisqPnzY3Zkq8QHh7vreYar"
access_token_secret = "XZn1rvLw10JnxJgKx05sW4eN0HqhaVjsasaqV5tEytsTu"
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)

api = tweepy.API(auth,wait_on_rate_limit=True)

In [53]:
tweetnum = 100

tbwTokenizer = TreebankWordTokenizer()
glove50Model = gensimapi.load("glove-wiki-gigaword-50")

tokenizedText = []
untokenizedText = ""
tknTextVector = []

def has_emoji(s):
    em_split_emoji = emoji.get_emoji_regexp().split(s)
    em_split_whitespace = [substr.split() for substr in em_split_emoji]
    em_split = functools.reduce(operator.concat, em_split_whitespace)
    emojiExists = False
    for emojiTest in em_split:
        if(emojiTest in UNICODE_EMOJI):
            emojiExists = True
    
    return emojiExists

def break_compound_word(compound_word,model):
    possible_words = []
    first_word=""
    for i,xchar in enumerate(compound_word):
        first_word+=xchar
        try:
            model[first_word]
            second_word = compound_word[i+1:] 
            model[second_word]
            possible_words.append([first_word,second_word])
        except:
            pass
        
    return possible_words

for tweet in tweepy.Cursor(api.search,q="#love",geocode="51.5098,-0.1180,1km",tweet_mode="extended").items(tweetnum):
    tweet_text = tweet._json["full_text"].lower()
    tweet_text_tokenized = tbwTokenizer.tokenize(tweet_text)
    tokenizedText.append(tweet_text_tokenized)
    tknVectorList = []
    
    for text in tweet_text_tokenized:
        try:
            wordVector = glove50Model[text]
            tknVectorList.append(wordVector)
        except:
            solutionFound = False
            if(text[-1] == "." or text[-2:-1] == ".." or text[-3:-1]=="..."):
                try:
                    try:
                        try:
                            ##Try getting words up o three ...
                            wordVector = glove50Model[text[:-3]]
                            tknVectorList.append(wordVector)
                            solutionFound = True
                        except:
                            pass
                        wordVector = glove50Model[text[:-2]]
                        tknVectorList.append(wordVector)
                        solutionFound = True
                    except:
                        pass
                    wordVector = glove50Model[text[:-1]]
                    tknVectorList.append(wordVector)
                    solutionFound = True
                except:
                    pass
            if(text[-1] == "…"):
                try:
                    wordVector = glove50Model[text[:-1]]
                    tknVectorList.append(wordVector)
                    solutionFound=True
                except:
                    pass
            if(has_emoji(text)):
                ##Try extracting text from strings containing both text and strings
                em_split_emoji = emoji.get_emoji_regexp().split(text)
                em_split_whitespace = [substr.split() for substr in em_split_emoji]
                em_split = functools.reduce(operator.concat, em_split_whitespace)
                for eachEmoji in em_split:
                    try:
                        emojiDetail = ud.name(eachEmoji)
                        emojiTokenized = tbwTokenizer.tokenize(emojiDetail.lower())
                        emojiWordVectors = []  
                        for descr in emojiTokenized:
                            emojiVector = glove50Model[descr]
                            emojiWordVectors.append(emojiVector)
                            
                        tknVectorList.extend(emojiWordVectors)
                        solutionFound = True
                    except:
                        pass
            child_words = break_compound_word(text,glove50Model)
            ##Try upto 5 combined words
            highestfrq = 0
            if(len(child_words)!= 0):
                for child_word_set in child_words:
                    both_word_freq = word_frequency(child_word_set[0],"en")*word_frequency(child_word_set[1],"en")
                    if (both_word_freq > highestfrq):
                        most_likely_combo = child_word_set
                        highestfrq = both_word_freq
            
                tknVectorList.append(glove50Model[most_likely_combo[0]])
                tknVectorList.append(glove50Model[most_likely_combo[1]])
                solutionFound = True
            
            if(not solutionFound):
                print("Failed to vectorize: " + text)
                print("    " + str(has_emoji(text)))
                    
    tknTextVector.extend(tknVectorList)

Failed to vectorize: louieandlola
    False
Failed to vectorize: //t.co/ieevpvgkyf
    False
Failed to vectorize: //t.co/io5e5rveh4
    False
Failed to vectorize: teddybeardress
    False
Failed to vectorize: oversizedjumperdress
    False
Failed to vectorize: yeezyboost
    False
Failed to vectorize: //t.co/cij1kz1sjz
    False
Failed to vectorize: //t.co/kknpl6fuva
    False
Failed to vectorize: couplegoals…
    False
Failed to vectorize: //t.co/cosbbs0vgg
    False
Failed to vectorize: beyourownkindofbeautiful
    False
Failed to vectorize: givelovetogetlove
    False
Failed to vectorize: flamingomediapr
    False
Failed to vectorize: //t.co/mcbk19f8py
    False
Failed to vectorize: beyourownkindofbeautiful
    False
Failed to vectorize: givelovetogetlove
    False
Failed to vectorize: flamingomediapr
    False
Failed to vectorize: //t.co/bnmgfynker
    False
Failed to vectorize: //t.co/s2fwrjd6dt
    False
Failed to vectorize: blackandwhite
    False
Failed to vectorize: motherandd

Failed to vectorize: beyourownkindofbeautiful
    False
Failed to vectorize: givelovetogetlove
    False
Failed to vectorize: flamingomediapr
    False
Failed to vectorize: //t.co/wzkqehuex8
    False
Failed to vectorize: //t.co/ewj3exsttn
    False
Failed to vectorize: homesweethome
    False
Failed to vectorize: //t.co/mpqhsu8fzq
    False
Failed to vectorize: ladysingstheblues————————————
    False
Failed to vectorize: //t.co/xlidpdsiyv
    False
Failed to vectorize: halfwaytoheaven
    False
Failed to vectorize: //t.co/aio12tb0cp
    False
Failed to vectorize: anyguycanbeanyguy
    False
Failed to vectorize: //t.co/i4jsi7z4xr
    False
Failed to vectorize: christmasiscoming
    False
Failed to vectorize: christmasdecorations…
    False
Failed to vectorize: //t.co/eafjwqayub
    False


In [10]:
def has_emoji(s):
    em_split_emoji = emoji.get_emoji_regexp().split(s)
    em_split_whitespace = [substr.split() for substr in em_split_emoji]
    em_split = functools.reduce(operator.concat, em_split_whitespace)
    emojiExists = False
    for emojiTest in em_split:
        if(emojiTest in UNICODE_EMOJI):
            emojiExists = True
    
    return emojiExists

stringtest = "💯🙌🏽❤️"

em_split_emoji = emoji.get_emoji_regexp().split(stringtest)
em_split_whitespace = [substr.split() for substr in em_split_emoji]
em_split = functools.reduce(operator.concat, em_split_whitespace)

em_split

['💯', '🙌🏽', '❤', '️']

In [25]:

ud.name("💯")

'HUNDRED POINTS SYMBOL'

In [37]:
import stanfordnlp

In [38]:
stanfordnlp.download('en')   # This downloads the English models for the neural pipeline
nlp = stanfordnlp.Pipeline() # This sets up a default neural pipeline in English
doc = nlp("Barack Obama was born in Hawaii.  He was elected president in 2008.")
doc.sentences[0].print_dependencies()

Using the default treebank "en_ewt" for language "en".
Would you like to download the models for: en_ewt now? (Y/n)
Y

Default download directory: /Users/tamimazmain/stanfordnlp_resources
Hit enter to continue or type an alternate directory.


Download location: /Users/tamimazmain/stanfordnlp_resources/en_ewt_models.zip


100%|██████████| 235M/235M [07:09<00:00, 546kB/s] 



Download complete.  Models saved to: /Users/tamimazmain/stanfordnlp_resources/en_ewt_models.zip
Extracting models file for: en_ewt
Cleaning up...Done.
Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/Users/tamimazmain/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/Users/tamimazmain/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/Users/tamimazmain/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/Users/tamimazmain/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: dep

../aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


In [43]:
# doc = nlp(["Barack","Obama","was", "born", "in","Hawaii",".","He","was","elected","president","in","2008","."])

doc = nlp("Barack Obama was born in Hawaii.  He was elected president in 2008.")
doc.sentences[0]

../aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


In [50]:
np.array(tknTextVector).shape

(4023, 50)

In [227]:
from sklearn.base import BaseEstimator, TransformerMixin

class HashtagDetailsExtractor(BaseEstimator, TransformerMixin):
    """
    Returns a list of tweets (in text form) from the given hashtag
    
    """
    def __init__(self):
        pass
    
    def fit(self,X,y=None):
        return self
    
    def transform(self,tag):
        tweet_text_list = []
        for tweet in tweepy.Cursor(api.search,q=tag,geocode="51.5098,-0.1180,1km",tweet_mode="extended").items(tweetnum):
            tweet_text_list.append(tweet._json["full_text"].lower())
        
        return tweet_text_list
   
class Tokenizer(BaseEstimator, TransformerMixin):
    """
    Returns tokenized text with the supplied Tokenizer 
    
    """
    def __init__(self,tokenizer_model,lowercase=True):
        self.tokenizer_model = tokenizer_model
        self.lowercase = lowercase
        
    def fit(self,X,y=None):
        return self
    
    def transform(self, X, y=None):
        tokenized_tweets = []
        for tweet in X:    
            tweet_text_tokenized = self.tokenizer_model.tokenize(tweet)
            tokenized_tweets.append(tweet_text_tokenized)
        return tokenized_tweets

class ConvertEmojis(BaseEstimator, TransformerMixin):
    """
    Extracts the description of the emoji and repalces it with the emoji itself
    
    """
    def __init__(self, tokenizer_model):
        self.tokenizer_model = tokenizer_model
    
    def fit(self,X,y=None):
        return self
    
    def hasemoji(self,s):
        em_split_emoji = emoji.get_emoji_regexp().split(s)
        em_split_whitespace = [substr.split() for substr in em_split_emoji]
        em_split = functools.reduce(operator.concat, em_split_whitespace)
        emojiExists = False
        for emojiTest in em_split:
            if(emojiTest in UNICODE_EMOJI):
                emojiExists = True
    
        return emojiExists
    
    def transform(self, X, y=None):
        X_copy = X.copy()
        for i,tweet in enumerate(X):
            shiftindex = 0
#             print(X[i])
            for ii,word_token in enumerate(tweet):
                if(self.hasemoji(word_token)):
                    em_split_emoji = emoji.get_emoji_regexp().split(word_token)
                    em_split_whitespace = [substr.split() for substr in em_split_emoji]
                    em_split = functools.reduce(operator.concat, em_split_whitespace)
                    emoji_detail_tokenized = []
                    for each_emoji in em_split:
                        try:
                            emoji_detail = ud.name(each_emoji)
                            emoji_tokenized = self.tokenizer_model.tokenize(emoji_detail.lower())                
                            emoji_detail_tokenized.extend(emoji_tokenized)
                        except:
#                             print("Attempted: " + word_token)
#                             print("Failed: " + each_emoji + " in:")
#                             print(emoji_tokenized)
                            print("FAIL")
#                             print(each_emoji)
                    print("Converted " + word_token + " to:")
                    print(emoji_detail_tokenized)
                    X_copy[i].pop(ii + shiftindex)
                    X_copy[i] = X_copy[i][:ii + shiftindex] + emoji_detail_tokenized + X_copy[i][ii + shiftindex:]
                    shiftindex += len(emoji_detail_tokenized) - 1
        return X_copy

class RemoveTrailingPeriods(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
    
    def fit(self, X ,y=None):
        return self
    
    def transform(self, X,y=None):
        for i,tweet in enumerate(X):
            indexes_with_period = []
            for ii, word_token in enumerate(tweet):
                while(X[i][ii][-1]=="." and len(X[i][ii]) != 1):
                    X[i][ii] = X[i][ii][:-1]
                
                if(X[i][ii][-1]=="…"):
                    X[i][ii] = X[i][ii][:-1]
                    
        
                    
        return X
                
class RemovePunctuation(BaseEstimator, TransformerMixin):
    
    def __init__(self,charsequence):
        self.charsequence = set(charsequence)
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_copy = X.copy()
        for i,tweet in enumerate(X):
            
            X_copy[i] = [text for text in tweet if not set([text]).issubset(self.charsequence)]
            
        return X_copy

    
class BreakupWords(BaseEstimator, TransformerMixin):
    
    def __init__(self,model):
        self.model = model
        
    def fit(self, X, y=None):
        return self
    
    def break_compound_word(self,compound_word):
        possible_words = []
        first_word=""
        for i,xchar in enumerate(compound_word):
            first_word+=xchar
            try:
                print("first_word")
                self.model[first_word]
                second_word = compound_word[i+1:]
                self.model[second_word]
                possible_words.append([first_word,second_word])
            except:
                pass
        
        return possible_words
    
    def transform(self, X, y=None):
        X_copy = X.copy()
        for i,tweet in enumerate(X):
            shiftindex = 0
            for ii, token in enumerate(tweet):
                child_words = self.break_compound_word(text)
                ##Try upto 5 combined words
                highestfrq = 0
                if(len(child_words)!= 0):
                    for child_word_set in child_words:
                        both_word_freq = word_frequency(child_word_set[0],"en")*word_frequency(child_word_set[1],"en")
                        if (both_word_freq > highestfrq):
                            most_likely_combo = child_word_set
                            highestfrq = both_word_freq
                    
                    X_copy[i].pop(ii + shiftindex)
                    X_copy[i] = X_copy[i][:ii + shiftindex] + most_likely_combo + X_copy[i][ii + shiftindex:]
                    shiftindex += len(most_likely_combo) - 1
                    
        return X_copy
                    
        
    
from nltk.tokenize import RegexpTokenizer

# treeBankTokenizer = RegexpTokenizer(r'\w+')
# treeBankTokenizer = TreebankWordTokenizer()
# pipelineTokenizer = Tokenizer(tokenizer_model=treeBankTokenizer)
# pipelineTagExtractor = HashtagDetailsExtractor()
# pipelinePeriod = RemoveTrailingPeriods()
# pipelineConvEmoji = ConvertEmojis(tokenizer_model=treeBankTokenizer)

# tokenized_tweets = pipelineTokenizer.fit_transform(pipelineTagExtractor.fit_transform("#love"))
# noperiod = pipelinePeriod.fit_transform(tokenized_tweets)

# emoji_reduced = pipelineConvEmoji.fit_transform(noperiod)

# pipelineRmPunc = RemovePunctuation([".","#",":","•",",","@","\"",";","\'"])

# punc_reduced = pipelineRmPunc.fit_transform(emoji_reduced)

# punc_reduced

word_set = [["deathbed","whyso","thedog"]]
# def break_compound_word1(compound_word,modell):
#     possible_words = []
#     first_word=""
#     for i,xchar in enumerate(compound_word):
#         first_word+=xchar
#         try:
#             modell[first_word]
#             second_word = compound_word[i+1:] 
#             modell[second_word]
#             possible_words.append([first_word,second_word])
#         except:
#             pass
        
#     return possible_words

children = break_compound_word1("thedog",glove50Model)

pipelineBrkWords = BreakupWords(glove50Model)
pipelineBrkWords.fit_transform(word_set)

[['deathbed', 'whyso', 'thedog']]

None


In [139]:
glove50Model["!"]

array([-0.58402 ,  0.39031 ,  0.65282 , -0.3403  ,  0.19493 , -0.83489 ,
        0.11929 , -0.57291 , -0.56844 ,  0.72989 , -0.56975 ,  0.53436 ,
       -0.38034 ,  0.22471 ,  0.98031 , -0.2966  ,  0.126   ,  0.55222 ,
       -0.62737 , -0.082242, -0.085359,  0.31515 ,  0.96077 ,  0.31986 ,
        0.87878 , -1.5189  , -1.7831  ,  0.35639 ,  0.9674  , -1.5497  ,
        2.335   ,  0.8494  , -1.2371  ,  1.0623  , -1.4267  , -0.49056 ,
        0.85465 , -1.2878  ,  0.60204 , -0.35963 ,  0.28586 , -0.052162,
       -0.50818 , -0.63459 ,  0.33889 ,  0.28416 , -0.2034  , -1.2338  ,
        0.46715 ,  0.78858 ], dtype=float32)

In [79]:
import pandas as pd
 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
 
# this is a very toy example, do not try this at home unless you want to understand the usage differences
docs=["the house had a tiny little mouse",
      "the cat saw the mouse",
      "the mouse ran away from the house",
      "the cat finally ate the mouse",
      "the end of the mouse story"
     ]
  
#instantiate CountVectorizer()
cv=CountVectorizer()
 
# this steps generates word counts for the words in your docs
word_count_vector=cv.fit_transform(docs)

word_count_vector.toarray()

array([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 2, 0],
       [0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 2, 0],
       [1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 2, 0]])

In [73]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

# print idf values
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"])
 
# sort ascending
df_idf.sort_values(by=['idf_weights'])

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']

In [91]:
ii = 1
shiftindex = 0
XXL = ["a", "b" , "k", "l", "x" , "j"]
print(XXL)
XXL.pop(ii + shiftindex)
print(XXL)
emoji_detail_tokenized = ["aa", "cc","zz"]
XXL = XXL[:ii + shiftindex] + emoji_detail_tokenized + XXL[ii + shiftindex:]
# shiftindex += len(emoji_detail_tokenized) - 1

print(XXL)

['a', 'b', 'k', 'l', 'x', 'j']
['a', 'k', 'l', 'x', 'j']
['a', 'aa', 'cc', 'zz', 'k', 'l', 'x', 'j']


In [93]:
ii = 2
shiftindex = 2
XXL = ['a', 'aa', 'cc', 'zz', 'k', 'l', 'x', 'j']
print(XXL)
XXL.pop(ii + shiftindex)
print(XXL)
emoji_detail_tokenized = ["jj", "kk","pp"]
XXL = XXL[:ii + shiftindex] + emoji_detail_tokenized + XXL[ii + shiftindex:]
# shiftindex += len(emoji_detail_tokenized) - 1

print(XXL)

['a', 'aa', 'cc', 'zz', 'k', 'l', 'x', 'j']
['a', 'aa', 'cc', 'zz', 'l', 'x', 'j']
['a', 'aa', 'cc', 'zz', 'jj', 'kk', 'pp', 'l', 'x', 'j']


In [126]:
# import unicodedata
# title="🥰"
# unicodedata.normalize('NFKD', title).encode('ascii','ignore')

"🥰".encode("utf-16","strict")

b'\xff\xfe>\xd8p\xdd'

In [209]:
ud.name("☺️")

TypeError: name() argument 1 must be a unicode character, not str

In [158]:
sett = set(["#",".","k"])

sett_1 = set(["#"])

sett_1.issubset(sett)

True

KeyError: "word 'don't' not in vocabulary"

In [206]:
def add(num1,num2):
    return num1 + num2

sample = 10
add(sample+=1,2)

SyntaxError: invalid syntax (<ipython-input-206-b4cdd51867c6>, line 5)